In [ ]:
import aliases

In [ ]:
_osjoin = _os.path.join

from everest.h5anchor import disk, globevars
from everest import classtools

In [ ]:
import ast as _ast
import pickle as _pickle
import os as _os

import h5py as _h5py

def resolve_eval(strn):
    out = _ast.literal_eval(strn)
    typout = type(out)
    if issubclass(typout, (list, tuple, frozenset)):
        return typout(resolve(sub) for sub in out)
    return out

stringresmeths = dict(
    _bytes_ = lambda x: pickle.loads(ast.literal_eval(x)),
    _eval_ = resolve_eval,
    _string_ = lambda x: x,
    )

def resolve_str(strn, /):
    for key, meth in stringresmeths.items():
        if strn.startswith(key):
            return meth(strn[len(key):])
    return strn

def resolve_attrs(attrs):
    return {key: resolve(attr) for key, attr in attrs.items()}

def resolve_dataset(dset):
    return dset[()]

def resolve_group(grp):
    out = {key: resolve(item) for key, item in grp.items()}
    out.update(resolve_attrs(grp.attrs))
    return out

resmeths = {
    str: resolve_str,
    _h5py.AttributeManager: resolve_attrs,
    _h5py.Dataset: resolve_dataset,
    _h5py.Group: resolve_group,
    }

def resolve(obj):
    for key, meth in resmeths.items():
        if isinstance(obj, key):
            return meth(obj)
    return obj

In [ ]:
def process_query(strn):
    split = strn.split('/')
    if len(split) > 1:
        for sub in split:
            yield from process_query(sub)
    elif isreg(strn):
        yield Reg(strn)
    elif isfnmatch(strn):
        yield FnMatch(strn)
    else:
        yield strn

class Readlet:
    __slots__ = ('reader', 'query')
    def __init__(self, reader, query, /):
        query = (process_query(query))
        self.reader, self.query = reader, query
    def read(self):
        with self.reader as target:
            for subquery in self.query:
                try:
                    target = subquery(target)
                except KeyError:
                    target = subquery(target.attrs)
            return resolve(target)
    def __getitem__(self, key):
        return self.reader[_osjoin(*self.query, key)]

@classtools.MROClassable
class Reader(disk.H5Manager):
    mroclasses = ('Readlet',)
    mode = 'r'
    Readlet = Readlet
    def __getitem__(self, query):
        return self.Readlet(self, query)

In [ ]:
tuple(process_query('a/b/c'))

In [ ]:
myreader = Reader('allout', aliases.datadir)

In [ ]:
out = myreader['aabaesweuh-luubrbriups']['inputs'].read()

In [ ]:
out

In [ ]:
myreader['aabaesweuh-luubrbriups']['inputs']['H'].read()

In [ ]:
%%timeit
myreader['aabaesweuh-luubrbriups']['inputs']['H'].read()

In [ ]:
with myreader as h5file:
    print(type(h5file['aabaesweuh-luubrbriups']['inputs']))

In [ ]:
myreadlet = myreader['aabaesweuh-luubrbriups']

In [ ]:
myreadlet.read()

In [ ]:
with myreader.open():
    print(len(myreader.h5file.keys()))

In [ ]:
with myreader as h5file:
    print(list(h5file.keys())[0]

In [ ]:
_h5py.AttributeManager

In [ ]:
with myreader as h5file:
    print(type(h5file['aabaesweuh-luubrbriups'].attrs))

In [ ]:
def myfn(a, b = 1, /, *, mode = 'b'):
    print(a, b, mode)

In [ ]:
myfn(1, 2)